# `PyLammpsMPI`

## Running on queueing systems

`pylammpsmpi` integrates with [dask distributed](https://distributed.dask.org/en/latest/) and [dask jobqueue](https://jobqueue.dask.org/en/latest/) to enable running lammps interactively on queueing systems. 

For this example, [slurm cluster](https://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html) provided by dask-jobqueue will be used to set up a cluster on which pylammpsmpi will be run.

In [1]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

Create a slurm cluster worker and ten cores. The optional would need to be tweaked according to the SLURM specifications.

In [2]:
cluster = SLURMCluster(queue='shorttime', cores=10, processes=1, job_cpu=10, memory="3GB", walltime="05:59:00")

Create a client and connect to the cluster

In [3]:
client = Client(cluster)
client

Client Scheduler: tcp://10.10.100.251:39269 Dashboard: http://10.10.100.251:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


By default the cluster has no workers. We add a worker,

In [4]:
cluster.scale(1)

A visual dashboard of jobs running on the cluster is also available. Once a cluster is set up, it can be provided to pylammpsmpi

In [5]:
from pylammpsmpi import LammpsLibrary

Specify two cores for the job(since we created a cluster with 2), choose the `mode` as `dask`, and pass the `client` to pylammpsmpi object

In [6]:
lmp = LammpsLibrary(cores=10, mode='dask', client=client)

The rest is similar as you would run on a local machine - except the calculations are run on the cluster.

Read an input file 

In [8]:
lmp.file("../tests/in.simple")

Check version of lammps

In [9]:
lmp.version

20200303

Check number of atoms

In [10]:
lmp.natoms

256

### Run commands

In [11]:
lmp.command("run 1")

In [12]:
lmp.command(["run 1", "run 1"])

Commands can also be direct

In [13]:
lmp.run(10)

In [14]:
lmp.mass(1, 20)

### Extract a global property

In [15]:
lmp.extract_global("boxxhi")

6.718384765530029

### Access thermo quantities

In [16]:
lmp.get_thermo("temp")

array(0.67569706)

Thermo quantities can also be accessed directly,

In [17]:
lmp.temp

array(0.67569706)

In [18]:
lmp.press

array(0.76594605)

### Accessing simulation box

In [19]:
lmp.extract_box()

([0.0, 0.0, 0.0],
 [6.718384765530029, 6.718384765530029, 6.718384765530029],
 0.0,
 0.0,
 0.0,
 [1, 1, 1],
 0)

### Accessing and changing atom properties

Get individual atom properties, for example force on each atoms

In [20]:
ff = lmp.gather_atoms("f")
print(type(ff))
print(len(ff))

<class 'numpy.ndarray'>
256


Get atom properties by their ids

In [21]:
ids = lmp.gather_atoms("id")

In [22]:
ff = lmp.gather_atoms("f", ids=ids[:10])
len(ff)

10

Change atom properties

In [23]:
ff = ff*0.5
lmp.scatter_atoms("f", ff, ids=ids[:10])

### Access value of variables

In [24]:
temp = lmp.extract_variable("tt", "all", 0)
temp

0.6756970578301337

### Access value of computes

In [25]:
ke = lmp.extract_compute("ke", 1, 1)
len(ke)

256

In [26]:
v = lmp.extract_compute("v", 1, 2, width=3)
v.shape

(256, 3)

In [27]:
lmp.extract_compute("1", 0, 0)

13.513941156602673

In [28]:
msd = lmp.extract_compute("msd", 0, 1, length=4)
msd[0]

0.011308113339734614

### Access values from fix

In [29]:
x = lmp.extract_fix("2", 0, 1, 1)
x

0.9745117319993072

### Change the simulation box

In [30]:
lmp.delete_atoms("group", "all")
lmp.reset_box([0.0,0.0,0.0], [8.0,8.0,8.0], 0.0,0.0,0.0)

Finally, the cluster is closed.

In [31]:
client.close()
cluster.close()